In [25]:
import requests
from shapely.geometry import Polygon, mapping
from datetime import datetime
import netrc
import getpass
from subprocess import PIPE, Popen
import os
import zipfile
import glob
import codecs
from osgeo import gdal

### Credentials setup for EarthData

In [3]:
ASF_USER = input("Enter Username: ")
ASF_PASS = getpass.getpass("Enter Password: ")

Enter Username:  pbillecocq
Enter Password:  ···············


### Function definitions

In [4]:
def build_bbox_string(polygon):
    '''
    Builds the string to include in the ASF search request. The bbox consists of 4 comma-separated numbers: lower left longitude,latitude, and upper right longitude,latitude.
    
    Parmeters
    ----------
    polygon: shapely polygon
    
    Returns
    ----------
    polygon_string : string
        String to include in the ASF request
    '''
    points = mapping(polygon)['coordinates'][0]
    lower_left = points[0]
    upper_right = points[2]
    bbox_string = f'{lower_left[0]},{lower_left[1]},{upper_right[0]},{upper_right[1]}'
        
    return bbox_string

def search_asf(platform, processingLevel, start, end, polygon, output_format):
    '''
    Search the ASF platform for images given the input parameters
    
    Parameters
    ----------
        platform : string
            Name of the imaging platform. Defaults to UAVSAR, but a list of supported platform is available on the ASF website
        processingLevel : string
            Processing level of the imaging product. 
            Possible values for UAVSAR : (KMZ, PROJECTED, PAULI, PROJECTED_ML5X5, STOKES, AMPLITUDE, BROWSE, COMPLEX, DEM_TIFF, PROJECTED_ML3X3, METADATA, AMPLITUDE_GRD, INTERFEROMETRY, INTERFEROMETRY_GRD, THUMBNAIL)
        start : datetime object
            Start date of the search period.
        end : datetime object
            End date of the search period.
        polygon : shapely polygon defining the Area of Interest,
        output_format: string
            Format being returned by the ASF API. Values : CSV, JSON, KML, METALINK, COUNT, DOWNLOAD, GEOJSON
        
    Returns
    -------
    Ouputs a search file
    '''
    base = 'https://api.daac.asf.alaska.edu/services/search/param'
    start_date = start.strftime('%Y-%m-%dT%H:%M:%SUTC')
    end_date = end.strftime('%Y-%m-%dT%H:%M:%SUTC')
    aoi_string = build_bbox_string(polygon)
    payload = {
        'platform': platform,
        'processingLevel': processingLevel,
        'start': start,
        'end': end,
        'bbox': aoi_string,
        'output': output_format
    }
    #request_string = build_asf_search_request(platform, processingLevel, start, end, polygon)
    # Perform actual request to the asf
    r = requests.get(base, params=payload)
    
    return r.json()

def download_single_product(productUrl, path):
    '''
    Downloading function.
    
    Paramters
    ----------
    productUrl: string
            Product download Url
    path: string
            path to download product to
            
    Returns
    -------
    Download the file
    '''
    process = Popen(['wget', productUrl, f'--user={ASF_USER}', f'--password={ASF_PASS}', '-P', path, '--progress=bar'], stderr=PIPE)
    started = False
    for line in process.stderr:
        line = line.decode("utf-8", "replace")
        if started:
            splited = line.split()
            if len(splited) == 9:
                percentage = splited[6]
                speed = splited[7]
                remaining = splited[8]
                print("Downloaded {} with {} per second and {} left.".format(percentage, speed, remaining), end='\r')
        elif line == os.linesep:
            started = True
    print(f"\nDone downloading {productUrl}")
    
def search_and_bulk_download(platform, processingLevel, start, end, polygon, path):
    '''
    Search the ASF platform for images given the input parameters. Download found products
    
    Parameters
    ----------
        platform : string
            Name of the imaging platform. Defaults to UAVSAR, but a list of supported platform is available on the ASF website
        processingLevel : string
            Processing level of the imaging product. 
            Possible values for UAVSAR : (KMZ, PROJECTED, PAULI, PROJECTED_ML5X5, STOKES, AMPLITUDE, BROWSE, COMPLEX, DEM_TIFF, PROJECTED_ML3X3, METADATA, AMPLITUDE_GRD, INTERFEROMETRY, INTERFEROMETRY_GRD, THUMBNAIL)
        start : datetime object
            Start date of the search period.
        end : datetime object
            End date of the search period.
        polygon : shapely polygon defining the Area of Interest,
        path: string
            Path to download product to
        '''
    results = search_asf(platform, processingLevel, start, end, polygon, output_format='JSON')[0]
    print(f'{len(results)} product(s) found')
    for result in results:
        downloadUrl = result['downloadUrl']
        download_single_product(downloadUrl, path)
        print('Unzipping product')
        with zipfile.ZipFile(path + result['fileName'], 'r') as zip_ref:
            zip_ref.extractall(path + result['productName'])
        print('Deleting original .zip')
        os.remove(path + result['fileName'])    

#### Processing function 

In [86]:
# folder is path to a folder with an .ann (or .txt) and .grd files (.amp1, .amp2, .cor, .unw, .int)

def uavsar_tiff_convert(path, verbose = False):
    """
    Builds a header file for the input UAVSAR .grd file,
    allowing the data to be read as a raster dataset.
    :param folder:   the folder containing the UAVSAR .grd and .ann files
    """

    # Empty lists to put information that will be recalled later.
    Lines_list = []
    Samples_list = []
    Latitude_list = []
    Longitude_list = []
    Files_list = []

    # Step 1: Look through folder and determine how many different flights there are
    # by looking at the HDR files.
    for files in os.listdir(path):
        if files [-4:] == ".grd":
            newfile = open(files[0:-4] + ".hdr", 'w')
            newfile.write("""ENVI
description = {DESCFIELD}
samples = NSAMP
lines = NLINE
bands = 1
header offset = 0
data type = DATTYPE
interleave = bsq
sensor type = UAVSAR L-Band
byte order = 0
map info = {Geographic Lat/Lon, 
            1.000, 
            1.000, 
            LON, 
            LAT,  
            0.0000555600000000, 
            0.0000555600000000, 
            WGS-84, units=Degrees}
wavelength units = Unknown
                """
                          )
            newfile.close()
            if files[0:18] not in Files_list:
                Files_list.append(files[0:18])

    #Variables used to recall indexed values.
    var1 = 0

    #Step 2: Look through the folder and locate the annotation file(s).
    # These can be in either .txt or .ann file types.
    for files in os.listdir(path):
        if Files_list[var1] and files[-4:] == ".txt" or files[-4:] == ".ann":
            #Step 3: Once located, find the info we are interested in and append it to
            # the appropriate list. We limit the variables to <=1 so that they only
            # return two values (one for each polarization of
            searchfile = codecs.open(path + files, encoding = 'windows-1252', errors='ignore')
            for line in searchfile:
                if "Ground Range Data Latitude Lines" in line:
                    Lines = line[65:70]
                    if verbose:
                        print(f"Number of Lines: {Lines}")
                    if Lines not in Lines_list:
                        Lines_list.append(Lines)

                elif "Ground Range Data Longitude Samples" in line:
                    Samples = line[65:70]
                    if verbose:
                        print(f"Number of Samples: {Samples}")
                    if Samples not in Samples_list:
                        Samples_list.append(Samples)

                elif "Ground Range Data Starting Latitude" in line:
                    Latitude = line[65:85]
                    if verbose:
                        print(f"Top left lat: {Latitude}")
                    if Latitude not in Latitude_list:
                        Latitude_list.append(Latitude)

                elif "Ground Range Data Starting Longitude" in line:
                    Longitude = line[65:85]
                    if verbose:
                        print(f"Top left Lon: {Longitude}")
                    if Longitude not in Longitude_list:
                        Longitude_list.append(Longitude)
    
                        
                 
            #Reset the variables to zero for each different flight date.
            var1 = 0
            searchfile.close()


    # Step 3: Open .hdr file and replace data for all type 4 (real numbers) data
    # this all the .grd files expect for .int
    for files in os.listdir(path):
        if files[-4:] == ".hdr":
            with open(files, "r") as sources:
                lines = sources.readlines()
            with open(files, "w") as sources:
                for line in lines:
                    if "data type = DATTYPE" in line:
                        sources.write(re.sub(line[12:19], "4", line))
                    elif "DESCFIELD" in line:
                        sources.write(re.sub(line[15:24], path, line))
                    elif "lines" in line:
                        sources.write(re.sub(line[8:13], Lines, line))
                    elif "samples" in line:
                        sources.write(re.sub(line[10:15], Samples, line))
                    elif "LAT" in line:
                        sources.write(re.sub(line[12:15], Latitude, line))
                    elif "LON" in line:
                        sources.write(re.sub(line[12:15], Longitude, line))
                    else:
                        sources.write(re.sub(line, line, line))
    
    # Step 3: Open .hdr file and replace data for .int file date type 6 (complex)                 
    for files in os.listdir(path):
        if files[-8:] == ".int.hdr":
            with open(files, "r") as sources:
                lines = sources.readlines()
            with open(files, "w") as sources:
                for line in lines:
                    if "data type = 4" in line:
                        sources.write(re.sub(line[12:13], "6", line))
                    elif "DESCFIELD" in line:
                        sources.write(re.sub(line[15:24], path, line))
                    elif "lines" in line:
                        sources.write(re.sub(line[8:13], Lines, line))
                    elif "samples" in line:
                        sources.write(re.sub(line[10:15], Samples, line))
                    elif "LAT" in line:
                        sources.write(re.sub(line[12:15], Latitude, line))
                    elif "LON" in line:
                        sources.write(re.sub(line[12:15], Longitude, line))
                    else:
                        sources.write(re.sub(line, line, line))
                        
    
    # Step 4: Now we have an .hdr file, the data is geocoded and can be loaded into python with rasterio
    # once loaded in we use gdal.Translate to convert and save as a .tiff
    
    data_to_process = glob.glob(path + '*.grd') # list all .grd files
    for data_path in data_to_process: # loop to open and translate .grd to .tiff, and save .tiffs using gdal
        raster_dataset = gdal.Open(data_path, gdal.GA_ReadOnly)
        print(data_path)
        print('****')
        print(raster_dataset)
        print('----')
        raster = gdal.Translate(data_path + '.tiff', raster_dataset, format = 'Gtiff', outputType = gdal.GDT_Float32)
    
    # Step 5: Save the .int raster, needs separate save because of the complex format
    data_to_process = glob.glob(path, '*.int.grd') # list all .int.grd files (only 1)
    for data_path in data_to_process:
        raster_dataset = gdal.Open(data_path, gdal.GA_ReadOnly)
        raster = gdal.Translate(os.path.join(path, os.path.basename(data_path) + '.tiff'), raster_dataset, format = 'Gtiff', outputType = gdal.GDT_CFloat32)

    print(".tiffs have been created")
    return

In [51]:
truc = gdal.Open('/tmp/silver_34715_20011-001_20016-002_0019d_s01_L090HH_01.cor.grd', gdal.GA_ReadOnly)
print(truc)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f48066f3ae0> >


In [87]:
troc = gdal.Open('/tmp/UA_GrMesa_08112_16095-004_17091-004_0313d_s01_L090_01/GrMesa_08112_16095-004_17091-004_0313d_s01_L090HH_01.cor.grd', gdal.GA_ReadOnly)
print(troc)

None


In [60]:
os.listdir('/tmp/')

['UA_grmesa_27416_20003-028_20017-006_0040d_s01_L090_01',
 'UA_grmesa_27416_20003-028_20013-004_0025d_s01_L090_01',
 'UA_grmesa_27416_20003-028_20008-004_0018d_s01_L090_01',
 'UA_grmesa_27416_20003-028_20005-007_0011d_s01_L090_01',
 '.empty']

### Testing playground

In [41]:
print('/home/jovyan/uavsar/toto/UA_grmesa_27416_20003-028_20005-007_0011d_s01_L090_01')
for folder in os.listdir('./toto'):
    uavsar_tiff_convert(f'/home/jovyan/uavsar/toto/{folder}/')

/home/jovyan/uavsar/toto/UA_grmesa_27416_20003-028_20005-007_0011d_s01_L090_01
/home/jovyan/uavsar/toto/UA_grmesa_27416_20003-028_20017-006_0040d_s01_L090_01/grmesa_27416_20003-028_20017-006_0040d_s01_L090HH_01.hgt.grd
****
None
----


ValueError: Received a NULL pointer.

In [81]:
# Gran Mesa polygon for testing purposes
gran_mesa_polygon = Polygon([(-108.2883, 38.882), (-108.0001, 38.882), (-108.0001, 39.0256), (-108.2883, 39.0256), (-108.2883, 38.882)])
start_date = datetime.strptime('2017-01-10 11:00:00', '%Y-%m-%d %H:%M:%S') 
end_date = datetime.strptime('2017-02-01 11:00:00', '%Y-%m-%d %H:%M:%S') 

In [82]:
result = search_asf(platform='UAVSAR', processingLevel='INTERFEROMETRY_GRD', start=start_date, end=end_date, polygon=gran_mesa_polygon, output_format='JSON')

In [83]:
len(result[0])

2

In [84]:
search_and_bulk_download(platform='UAVSAR', processingLevel='INTERFEROMETRY_GRD', start=start_date, end=end_date, polygon=gran_mesa_polygon, path='/tmp/')

2 product(s) found
Downloaded 99% with 187M per second and 0s left....
Done downloading https://datapool.asf.alaska.edu/INTERFEROMETRY_GRD/UA/GrMesa_26108_16095-005_17091-005_0313d_s01_L090_01_int_grd.zip
Unzipping product
Deleting original .zip
Downloaded 99% with 14.1M per second and 0s left...
Done downloading https://datapool.asf.alaska.edu/INTERFEROMETRY_GRD/UA/GrMesa_08112_16095-004_17091-004_0313d_s01_L090_01_int_grd.zip
Unzipping product
Deleting original .zip
